In [1]:
import random 
import efc as efc
import numpy as np
import pandas as pd
import seaborn as sns

from efc import EnergyBasedFlowClassifier
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

verbose = False
%matplotlib inline

/home/leohmelo/venv/lib/python3.10/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/leohmelo/venv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/leohmelo/venv/lib/python3.10/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/leohmelo/venv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
silos_path = "./datasets/silos/"
silos_files = [f for f in listdir(silos_path) if isfile(join(silos_path, f))]

silos = {}
for silo in silos_files:
    silo_name = silo.replace(".csv.gz", "")
    silos[silo_name] = pd.read_csv(silos_path + silo)
    
    if(verbose):
        print(f"{silo_name} >>> \n{silos[silo_name]['Label'].value_counts()}")

In [3]:
def preprocess(X):
    cols = X.select_dtypes(include=[np.float64]).columns
    X[cols] = X[cols].astype(np.float32)

    X.fillna(0, inplace=True)
    X.replace(np.inf, 9999, inplace=True)
    
    return X

In [4]:
def average(lst):
    return sum(lst) / len(lst)

In [5]:
balanced = True 
def rebalance_agent(agent):
    agent_label_count = agent["Label"].value_counts().to_dict()
    benign_count = agent_label_count[0] 
    malicious_count = agent_label_count[1]
    
    if(balanced):
        if (benign_count > malicious_count):
            downsampled_0 = resample(
                                agent[agent.Label == 0], 
                                replace=False, 
                                n_samples=agent[agent.Label == 1].shape[0],
                                random_state=1337
                            )
            agent_balanced = pd.concat([downsampled_0, agent[agent.Label == 1]])
        elif (benign_count < malicious_count): 
            downsampled_1 = resample(
                                agent[agent.Label == 1], 
                                replace=False, 
                                n_samples=agent[agent.Label == 0].shape[0], 
                                random_state=1337
                            )
            agent_balanced = pd.concat([downsampled_1, agent[agent.Label == 0]])
        else:
            agent_balanced = agent
    else:
        agent_balanced = agent
    
    return agent_balanced

In [6]:
features_to_delete = [
     'DST_TO_SRC_SECOND_BYTES',
     'FTP_COMMAND_RET_CODE',
     'RETRANSMITTED_IN_PKTS',
     'RETRANSMITTED_IN_BYTES',
     'DNS_TTL_ANSWER',
     'SERVER_TCP_FLAGS',
     'DNS_QUERY_TYPE',
     'DNS_QUERY_ID',
     'SRC_TO_DST_SECOND_BYTES',
     'Unnamed: 0',
     'SRC_TO_DST_AVG_THROUGHPUT',
     'MIN_TTL',
     'SHORTEST_FLOW_PKT',
     'IN_BYTES',
     'MAX_TTL',
     'LONGEST_FLOW_PKT',
     'MAX_IP_PKT_LEN',
     'RETRANSMITTED_OUT_PKTS',
     'NUM_PKTS_128_TO_256_BYTES',
     'OUT_PKTS',
     'RETRANSMITTED_OUT_BYTES',
     'MIN_IP_PKT_LEN',
     'NUM_PKTS_256_TO_512_BYTES',
     'DURATION_IN',
     'Label'
]

In [ ]:
siloed_feat_efc = {}

for key, silo in silos.items():
    
    X = silo.drop(columns=features_to_delete).copy()
    X = preprocess(X)
    y = silo["Label"].copy()
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=42, stratify=y, shuffle=True, test_size=0.3
    )
    
    X_train["Label"] = y_train 
    X_train = rebalance_agent(X_train) 
    y_train = X_train["Label"] 
    X_train = X_train.drop(columns="Label").copy() 
    
    # train and test EFC
    clf = EnergyBasedFlowClassifier(cutoff_quantile=0.95)
    print(y_train.value_counts())
    clf.fit(X_train, y_train)
    
    siloed_feat_efc[key] = clf
    
    y_pred, y_energies = clf.predict(X_test, return_energies=True)
    
    print(key, "| cutoff =", clf.estimators_[0].cutoff_, "\n-------------------------------------------------------------------\n", classification_report(y_test, y_pred))
    
    # ploting energies
    benign = np.where(y_test == 0)[0]
    malicious = np.where(y_test == 1)[0]

    benign_energies = y_energies[benign]
    malicious_energies = y_energies[malicious]
    cutoff = clf.estimators_[0].cutoff_

    bins = np.histogram(y_energies, bins=60)[1]

    plt.hist(
        malicious_energies,
        bins,
        facecolor="#006680",
        alpha=0.7,
        ec="white",
        linewidth=0.3,
        label="malicious",
    )
    plt.hist(
        benign_energies,
        bins,
        facecolor="#b3b3b3",
        alpha=0.7,
        ec="white",
        linewidth=0.3,
        label="benign",
    )
    plt.axvline(cutoff, color="r", linestyle="dashed", linewidth=1)
    plt.legend()

    plt.xlabel("Energy", fontsize=12)
    plt.ylabel("Density", fontsize=12)

    plt.show()
    

In [ ]:
data_feat = {}
full_feat = []
avg_feat_info = {} 

for model_key, model in siloed_feat_efc.items():
    
    silos_y_test = np.array([]) 
    silos_y_pred = np.array([]) 
    
    for key, silo in silos.items():
        #if model_key == key:
        #    continue
            
        X = silo.drop(columns=features_to_delete).copy()
        #X.dropna(inplace=True)
        X = preprocess(X)
        y = silo["Label"].copy()
    
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, random_state=42, stratify=y, shuffle=True, test_size=0.3
        )
        
        y_pred = model.predict(X_test)
        score = f1_score(y_test, y_pred)
        
        silos_y_test = np.concatenate((silos_y_test, y_test), axis=None)
        silos_y_pred = np.concatenate((silos_y_pred, y_pred), axis=None)
        
        data_feat[key] = score
    
    print(f"{model_key} >>> ")
    print(confusion_matrix(silos_y_test, silos_y_pred))
        
    full_feat.append(data_feat)
    avg_feat_info[model_key] = data_feat
    data_feat = {}

In [ ]:
cross_silo_feat_efc = pd.DataFrame(full_feat, index=list(siloed_feat_efc.keys()))

In [ ]:
plt.figure(figsize = (20,16))
ax = sns.heatmap(cross_silo_feat_efc, xticklabels=cross_silo_feat_efc.columns, yticklabels=cross_silo_feat_efc.columns, annot=True)

In [ ]:
# Calculo da média do modelo de cada um dos agentes avaliado em outros datasets
average_f1_feat_score = dict()

for i in range(len(avg_feat_info)):
    name = "agent" + str(i+1)
    average_f1_feat_score[name] = average(avg_feat_info[name].values())

for name, value in average_f1_feat_score.items():
    print(value)

## Validação entre os silos utilizando feature selection

In [ ]:
agents = silos.copy()
seed = 42

### FedAvg hyperparameters

In [ ]:
rounds = 50

# Epochs
# FedAvg requires a minimum number of training epochs before averaging
# For epoch=1 is the same as FedSGD (https://www.cs.cornell.edu/~shmat/shmat_ccs15.pdf)
epochs = 20

# train_size same as batch size
# If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the train split. 
# If int, represents the absolute number of train samples
train_size=100 #0.05

remove_agents = []

### Auxiliary Functions

In [ ]:
def process_results(results, remove_list=[]):
    all_values = []
    for major_key in results.keys():
        if (major_key in remove_list) in remove_list:
            continue
        values = []
        for key in agents.keys():
            if key in remove_list:
                continue
            values.append(results[major_key][key])
        all_values.append(values)
    return all_values

In [ ]:
# update each agent model by current global model values
def load_global_model(model):
    model.intercept_ = fedavg.intercept_.copy()
    model.coef_ = fedavg.coef_.copy()
    return model

In [ ]:
def update_global_model(applicable_models, round_weights):
    # Average models parameters
    coefs = []
    intercept = []
    for model in applicable_models:
        coefs.append(model.coef_)
        intercept.append(model.intercept_)
    
    # average and update FedAvg (aggregator model)
    fedavg.coef_ = np.average(coefs, axis=0, weights=round_weights) # weight
    fedavg.intercept_ = np.average(intercept, axis=0, weights=round_weights) # weight

In [ ]:
def train_local_agent(X, y, model, epochs, class_weight):
    for _ in range(0, epochs):
        model.partial_fit(X, y, classes=np.unique(y), sample_weight=class_weight)
    return model

In [ ]:
# update each agent model by current global model values
def load_global_model(model):
    model.intercept_ = fedavg.intercept_.copy()
    model.coef_ = fedavg.coef_.copy()
    return model

In [ ]:
def update_global_model(applicable_models, round_weights):
    # Average models parameters
    coefs = []
    intercept = []
    for model in applicable_models:
        coefs.append(model.coef_)
        intercept.append(model.intercept_)
    
    # average and update FedAvg (aggregator model)
    fedavg.coef_ = np.average(coefs, axis=0, weights=round_weights) # weight
    fedavg.intercept_ = np.average(intercept, axis=0, weights=round_weights) # weight

In [ ]:
def train_local_agent(X, y, model, epochs, class_weight):
    for _ in range(0, epochs):
        model.partial_fit(X, y, classes=np.unique(y), sample_weight=class_weight)
    return model

### Global Model

In [ ]:
fedavg = SGDClassifier() 
lr = 0.1
def initialize_global_model(agent, features_to_drop):
    lr = 0.1
    fedavg = SGDClassifier(n_jobs=-1, random_state=seed, loss="log", learning_rate='constant', eta0=lr, verbose=0) # global
    features = agent.drop(columns=features_to_drop).shape[1] # total number of features
    fedavg.intercept_ = np.zeros(1)
    fedavg.coef_ = np.zeros((1, features))
    fedavg.classes_ = np.array([0, 1])
    return fedavg

In [ ]:
sgd_feature = dict()
for agent in agents.keys():
    sgd_feature[agent] = SGDClassifier(n_jobs=-1, random_state=seed, loss="log", learning_rate='constant', eta0=lr, verbose=0)

In [ ]:
fedavg = initialize_global_model(agents["agent1"], features_to_delete)

### Benign Samples

In [ ]:
rounds_f1_score = dict() 

print("Federated Training Started.")
for r in range(rounds):
    sample_size = random.randint(int(len(agents)/2), len(agents))
    applicable_agents = random.sample(list(agents.keys()), k=sample_size) # select random agents/silos

    applicable_models = []
    applicable_name = []

    round_weights = []
    dataset_size = 0

    for name, agent in agents.items():
        if name not in applicable_agents: # check if agent/silos is in applicable_agents
            continue

        if name in remove_agents: # check if agent/silos is in remove_agents, if yes, jump (refer to FedAvg hyperparams)
            continue
        
        prep = StandardScaler() 
        applicable_name.append(name)

        agent_balanced = rebalance_agent(agent)
        
        print(".", end=" ")
        X = agent_balanced.drop(columns=features_to_delete)
        y = agent_balanced.Label
      
        X = preprocess(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=train_size, stratify=y) #stratified / random_state=seed
        X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]

        sample_weights = compute_sample_weight('balanced', y=y_train)
        sgd_feature[name] = load_global_model(sgd_feature[name])
        sgd_feature[name] = train_local_agent(X_train, y_train, sgd_feature[name], epochs, sample_weights)
        round_weights.append(X_train.shape[0])
        applicable_models.append(sgd_feature[name])

    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size

    try:
        update_global_model(applicable_models, round_weights)
    except:
        print("Error updating global model, due to division by 0")
    
    for name, agent in agents.items():
        sgd_feature[name] = load_global_model(sgd_feature[name])

In [ ]:
cross_validation_results = dict()

print("Cross Validation Analisys Started...")
for name, agent in agents.items():
    
    silos_y_test = np.array([]) 
    silos_y_pred = np.array([])
    
    print(".", end="")
    if name in remove_agents:
            continue
            
    agent_balanced = rebalance_agent(agent) 
    
    X = agent_balanced.drop(columns=features_to_delete)
    y = agent_balanced.Label

    X = preprocess(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y) #stratified
    
    X_train = prep.fit_transform(X_train)
    X_test = prep.transform(X_test) 
    
    y_pred = sgd_feature[name].predict(X_test)
    
    cross_validation_results[name] = dict()
    cross_validation_results[name][name] = f1_score(y_test, y_pred, average="binary")
    
    print(name + " " + str(cross_validation_results[name][name]))
    
    silos_y_test = np.concatenate((silos_y_test, y_test), axis=None)
    silos_y_pred = np.concatenate((silos_y_pred, y_pred), axis=None)
    
    for test_name, test_agent in agents.items():
        if test_name == name:
            continue
        
        X = test_agent.drop(columns=features_to_delete)
        y = test_agent.Label
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y) #stratified
        X_test = preprocess(X_test)
        X_test = prep.transform(X_test)
        
        y_pred = sgd_feature[name].predict(X_test)
        cross_validation_results[name][test_name] = f1_score(y_test, y_pred, average="binary")
        
        silos_y_test = np.concatenate((silos_y_test, y_test), axis=None)
        silos_y_pred = np.concatenate((silos_y_pred, y_pred), axis=None)
    
    print(f"{name} >>> ")
    print(confusion_matrix(silos_y_test, silos_y_pred))

In [ ]:
feature_results = process_results(cross_validation_results, remove_agents)

In [ ]:
labels = [i for i in agents.keys() if i not in remove_agents]
plt.figure(figsize = (30,15))
ax = sns.heatmap(feature_results, vmin=0, vmax=1, annot=True, fmt=".4f", xticklabels=labels, yticklabels=labels, cmap="YlGnBu")

In [ ]:
average_f1_score = dict()
for i, name in enumerate(agents.keys()):
    average_f1_score[name] = average(feature_results[i])

for value in average_f1_score.values():
    print(value)

# Evaluation models by combining EFC + Federated Learning Model

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

Combination of the models using **OR** operation 

In [ ]:
cross_validation_results = dict()

print("Cross Validation Analisys Started...")
for name, agent in agents.items():
    
    silos_y_test = np.array([]) 
    silos_y_pred = np.array([])
    
    print(".", end="")
    if name in remove_agents:
            continue
            
    agent_balanced = rebalance_agent(agent) 
    
    X = agent_balanced.drop(columns=features_to_delete)
    y = agent_balanced.Label

    X = preprocess(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y) #stratified
    
    X_train = prep.fit_transform(X_train)
    X_test = prep.transform(X_test) 
    
    y_pred_federated = sgd_feature[name].predict(X_test)
    y_pred_efc = siloed_feat_efc[name].predict(X_test) 
    
    y_pred = np.logical_or(y_pred_federated, y_pred_efc)
    
    silos_y_test = np.concatenate((silos_y_test, y_test), axis=None)
    silos_y_pred = np.concatenate((silos_y_pred, y_pred), axis=None)
    
    cross_validation_results[name] = dict()
    cross_validation_results[name][name] = f1_score(y_test, y_pred, average="binary")
    
    print(name + " " + str(cross_validation_results[name][name]))
    
    for test_name, test_agent in agents.items():
        if test_name == name:
            continue
        
        X = test_agent.drop(columns=features_to_delete)
        y = test_agent.Label
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y) #stratified
        X_test = preprocess(X_test)
        X_test = prep.transform(X_test)
        
        y_pred_federated = sgd_feature[name].predict(X_test)
        y_pred_efc = siloed_feat_efc[name].predict(X_test)
        
        y_pred = np.logical_or(y_pred_federated, y_pred_efc)
        
        silos_y_test = np.concatenate((silos_y_test, y_test), axis=None)
        silos_y_pred = np.concatenate((silos_y_pred, y_pred), axis=None)
        
        cross_validation_results[name][test_name] = f1_score(y_test, y_pred, average="binary")
    
    print(f"{name} >>> ")
    cm = confusion_matrix(silos_y_test, silos_y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()

    

In [ ]:
array_valores_01 = np.array([0,0,0])
array_valores_02 = np.array([1,0,0])

print(np.logical_or(array_valores_01, array_valores_02))

In [ ]:
feature_results = process_results(cross_validation_results, remove_agents)

In [ ]:
labels = [i for i in agents.keys() if i not in remove_agents]
plt.figure(figsize = (30,15))
ax = sns.heatmap(feature_results, vmin=0, vmax=1, annot=True, fmt=".4f", xticklabels=labels, yticklabels=labels, cmap="YlGnBu")

In [ ]:
average_f1_score = dict()
for i, name in enumerate(agents.keys()):
    average_f1_score[name] = average(feature_results[i])

for value in average_f1_score.values():
    print(value)

Combination of the models using **AND** operation 

In [ ]:
cross_validation_results = dict()

print("Cross Validation Analisys Started...")
for name, agent in agents.items():
    
    silos_y_test = np.array([]) 
    silos_y_pred = np.array([])
    
    print(".", end="")
    if name in remove_agents:
            continue
            
    agent_balanced = rebalance_agent(agent) 
    
    X = agent_balanced.drop(columns=features_to_delete)
    y = agent_balanced.Label

    X = preprocess(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y) #stratified
    
    X_train = prep.fit_transform(X_train)
    X_test = prep.transform(X_test) 
    
    y_pred_federated = sgd_feature[name].predict(X_test)
    y_pred_efc = siloed_feat_efc[name].predict(X_test) 
    
    y_pred = np.logical_and(y_pred_federated, y_pred_efc)
    
    silos_y_test = np.concatenate((silos_y_test, y_test), axis=None)
    silos_y_pred = np.concatenate((silos_y_pred, y_pred), axis=None)

    cross_validation_results[name] = dict()
    cross_validation_results[name][name] = f1_score(y_test, y_pred, average="binary")
    
    print(name + " " + str(cross_validation_results[name][name]))
    
    for test_name, test_agent in agents.items():
        if test_name == name:
            continue
        
        X = test_agent.drop(columns=features_to_delete)
        y = test_agent.Label
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y) #stratified
        X_test = preprocess(X_test)
        X_test = prep.transform(X_test)
        
        y_pred_federated = sgd_feature[name].predict(X_test)
        y_pred_efc = siloed_feat_efc[name].predict(X_test)
        
        y_pred = np.logical_and(y_pred_federated, y_pred_efc)
        
        silos_y_test = np.concatenate((silos_y_test, y_test), axis=None)
        silos_y_pred = np.concatenate((silos_y_pred, y_pred), axis=None)
        
        cross_validation_results[name][test_name] = f1_score(y_test, y_pred, average="binary")
    
    print(f"{name} >>> ")
    cm = confusion_matrix(silos_y_test, silos_y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()


In [ ]:
feature_results = process_results(cross_validation_results, remove_agents)

In [ ]:
labels = [i for i in agents.keys() if i not in remove_agents]
plt.figure(figsize = (30,15))
ax = sns.heatmap(feature_results, vmin=0, vmax=1, annot=True, fmt=".4f", xticklabels=labels, yticklabels=labels, cmap="YlGnBu")

In [ ]:
average_f1_score = dict()
for i, name in enumerate(agents.keys()):
    average_f1_score[name] = average(feature_results[i])

for value in average_f1_score.values():
    print(value)